# Satellite Entry/Exit Event Detection with Ground Sensor Field of View (Rectangular FoV)

The FoV of the ground sensor is represented as a **Rectangular** shape. The sensor's attitude can be adjusted to deviate from the zenith orientation by specifying three consecutive rotation angles. If no specific attitude adjustment is needed, these rotation angles should be set to 0 degrees.

Additionally, the code includes three detectors:
- GroundFieldOfViewDetector
- ElevationDetector
- BooleanDetector

In [3]:
import orekit
vm = orekit.initVM()

from orekit.pyhelpers import setup_orekit_curdir
setup_orekit_curdir()

from org.orekit.attitudes import NadirPointing, LofOffset
from org.orekit.bodies import OneAxisEllipsoid, GeodeticPoint
from org.orekit.frames import FramesFactory, LOFType, TopocentricFrame, Frame, Transform
from org.orekit.time import TimeScalesFactory, AbsoluteDate
from org.orekit.utils import Constants, IERSConventions, PVCoordinatesProvider
from org.orekit.propagation import SpacecraftState
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator
from org.orekit.models.earth.tessellation import EllipsoidTessellator
from org.orekit.propagation.events import EventDetector, EventsLogger, ElevationDetector, GroundFieldOfViewDetector, BooleanDetector
from org.orekit.propagation.events.handlers import ContinueOnEvent, StopOnEvent
from org.orekit.geometry.fov import DoubleDihedraFieldOfView

from org.hipparchus.geometry.euclidean.threed import Vector3D, Line, RotationOrder, Rotation, RotationConvention
from org.hipparchus.geometry.spherical.twod import SphericalPolygonsSet

from math import radians, degrees, pi, sqrt, atan
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

First call the frames and time scales to be used in the code and create Earth body.

In [5]:
UTC = TimeScalesFactory.getUTC()                               # Define UTC time scale.
ECI = FramesFactory.getEME2000()                               # Define ECI reference frame.
ECEF = FramesFactory.getITRF(IERSConventions.IERS_2010, True)  # Define ECEF reference frame.
TEME = FramesFactory.getTEME()                                 # Define TEME reference frame. 
ITRF = ECEF

R_earth  = Constants.WGS84_EARTH_EQUATORIAL_RADIUS             # Radius of earth
Mu_earth = Constants.WGS84_EARTH_MU                            # Gravitational parameter of earth
f_earth  = Constants.WGS84_EARTH_FLATTENING                    # Earth flattening value

earth = OneAxisEllipsoid(R_earth, f_earth, ITRF)               # Create earth here.

Here, we first take TLE as an input, and from the *TLEPropagator* method we recieve its PV to get Kepler Elements.

In [7]:
# ISS 25544
tle_line1 = "1 25544U 98067A   24265.15389185  .00020992  00000-0  38244-3 0  9995"
tle_line2 = "2 25544  51.6375 200.0394 0007352  15.0244 345.0962 15.49404240473433"
mytle = TLE(tle_line1, tle_line2)


initialDate = mytle.getDate()                    # This is the TLE epoch date read from first line.
finalDate = initialDate.shiftedBy(3600.0 * 72)   # Shift initial date by x hours.

print('Propagation initial date: ', initialDate)
print('Propagation final date: ', finalDate)

satellite_mass = 466615.0                        # Satellite mass [kg]
attitudeProvider = NadirPointing(TEME, earth)    # Also provide attitude of the satellite

SGP4 = TLEPropagator.selectExtrapolator(mytle, attitudeProvider, satellite_mass) 

Propagation initial date:  2024-09-21T03:41:36.25584Z
Propagation final date:  2024-09-24T03:41:36.25584Z


Geodetic coordinates (lat/long/alt) of the ground station is defined below. Coordinate frame of the station is accepted as *TopocentricFrame*. This frame is associated to a position near the surface of a body shape.

The origin of the frame is at the defined geodetic point location, and the right-handed canonical trihedra is:

- X axis in the local horizontal plane (normal to zenith direction) and following the local parallel towards East
- Y axis in the horizontal plane (normal to zenith direction) and following the local meridian towards North
- Z axis towards Zenith direction

In [9]:
# Define the ground station location
latitude  = input('Latitude (degree):  ') 
latitude  = radians(float(latitude))
longitude = input('Longitude (degree):  ')
longitude = radians(float(longitude))
altitude  = input('Altitude (meter):  ')
altitude  = (float(altitude))

# Frame of the ground station is given as Topocentric Frame.
station_gp = GeodeticPoint(latitude, longitude, altitude)
station_frame = TopocentricFrame(earth, station_gp, "topo")

Latitude (degree):   32
Longitude (degree):   35
Altitude (meter):   0


In the following parts, we are going to create ground sensor with different FoV shapes using related lat/long/alt coordinates.

In the following parts, we are going to create ground sensor with different FoV shapes using related lat/long/alt coordinates.

### Rotations with Respective Axes

The frame of the ground station defined as topocentric previously. Considering the definition, user may rotate this frame to get the desired attitude for the ground sensor. 

In this code, **rotation is done in X-Y-Z order** respectively. This means that the topocentric frame defined previously is first rotated around X axis, then Y axis, then Z axis. Rotation order may be changed, but it must be bear in mind that having the first rotation around Z axis is prone to errors. 

In [14]:
# Using MINUS for X, Y and Z axes in this code. Our notation becomes:
# (X = East, Y = North, Z = Zenith)
# which is the definition of topocentric frame in Orekit.
rotation1 = input('Enter the first rotation angle in degrees:  ')
rotation1 = radians(float(rotation1))
rotation2 = input('Enter the second rotation angle in degrees:  ')
rotation2 = radians(float(rotation2))
rotation3 = input('Enter the third rotation angle in degrees:  ')
rotation3 = radians(float(rotation3))

# If rotation vectors are entered PLUS instead of MINUS, then the correct XYZ notation becomes:
# (X = West, Y = South, Z = Zenith) ?? (not sure but it becomes incorrect)

# First rotation
rotation = Transform(AbsoluteDate.ARBITRARY_EPOCH, Rotation(Vector3D.MINUS_I, rotation1, RotationConvention.VECTOR_OPERATOR))
myFoVFrame = Frame(station_frame, rotation, "myFovFrame")
# Second rotation
rotation = Transform(AbsoluteDate.ARBITRARY_EPOCH, Rotation(Vector3D.MINUS_J, rotation2, RotationConvention.VECTOR_OPERATOR))
myFoVFrame = Frame(myFoVFrame, rotation, "myFovFrame")
# Third rotation
rotation = Transform(AbsoluteDate.ARBITRARY_EPOCH, Rotation(Vector3D.MINUS_K, rotation3, RotationConvention.VECTOR_OPERATOR))
myFoVFrame = Frame(myFoVFrame, rotation, "myFovFrame")

Enter the first rotation angle in degrees:   0
Enter the second rotation angle in degrees:   0
Enter the third rotation angle in degrees:   0


After rotating our initial topocentric frame three times, we reach our desired attitude for the sensor attached to the ground station.

### Creating the Ground FoV Sensor 

Defining FoV shape of the ground sensor and creating *GroundFieldOfViewDetector* together with *ElevationDetector* method. Finally these event detectors are combined with a *BooleanDetector*, and they are added to propagator for monitoring.

In [18]:
# Rectangular Field-Of-View
verticalfov  = input('Enter the vertical half-angle (in degrees) of the cone formed by the target and the ground detector: ')
verticalfov  = float(verticalfov)
horizontalfov  = input('Enter the horizontal half-angle (in degrees) of the cone formed by the target and the ground detector: ')
horizontalfov  = float(horizontalfov)
fov_margin = 0.0

# Create the Field-Of-View 
imagerFOV = DoubleDihedraFieldOfView(Vector3D.PLUS_K,        # ← center
                                     Vector3D.PLUS_I,        # ← axis1
                                     radians(verticalfov),   # ← halfAperture1
                                     Vector3D.PLUS_J,        # ← axis2
                                     radians(horizontalfov), # ← halfAperture2
                                     fov_margin)             # ← margin

# Define Ground Field of View Detector
GFoVDetector = GroundFieldOfViewDetector(myFoVFrame, imagerFOV).withMaxCheck(5.0).withHandler(ContinueOnEvent())
GFoV_logger = EventsLogger()

# Define Elevation Detector
horizon_limit = radians(0.0)
eleDetector = ElevationDetector(station_frame).withConstantElevation(horizon_limit).withHandler(ContinueOnEvent())
ele_logger = EventsLogger()

#combined = BooleanDetector.andCombine(BooleanDetector.notCombine(GFoVDetector),eleDetector)
combinedDetector = BooleanDetector.andCombine([BooleanDetector.notCombine(GFoVDetector),eleDetector])
combined_logger = EventsLogger()

# Add the seperate detectors to the SGP4 propagator
SGP4.addEventDetector(GFoV_logger.monitorDetector(GFoVDetector))
SGP4.addEventDetector(ele_logger.monitorDetector(eleDetector))
SGP4.addEventDetector(combined_logger.monitorDetector(combinedDetector))

Enter the vertical half-angle (in degrees) of the cone formed by the target and the ground detector:  25
Enter the horizontal half-angle (in degrees) of the cone formed by the target and the ground detector:  25


Now start propagating the TLE with SGP4. Here, propagation interval has to be sufficiently small because if it is large, detectors may miss certain events.

In [20]:
pos = []                                                       # position vector array to be filled.
SGP4 = PVCoordinatesProvider.cast_(SGP4)
###Start SGP4 propagation from initialDate up until finalDate
while (initialDate.compareTo(finalDate) <= 0.0):
    SGP4_pv = SGP4.getPVCoordinates(initialDate, ITRF)         # Get PV coordinates
    posSGP4 = SGP4_pv.getPosition()                            # But we only want position vector.
    pos.append((posSGP4.getX(),posSGP4.getY(),posSGP4.getZ())) # Get individual elements of position
    posSGP4 = pos
    initialDate = initialDate.shiftedBy(10.0)                  # Propagate with 10 sec intervals.

Finally the logged events are retrieved and printed in a readable way.

In [22]:
gfov_events = GFoV_logger.getLoggedEvents()            

entry_list = []
exit_list = []
duration_list = []

for event in gfov_events: 
    if not event.isIncreasing():
        gfov_entry = event.getState().getDate()
        entry_str = str(gfov_entry).replace("T", " ==> ")
        entry_list.append(entry_str)
    else:
        gfov_exit = event.getState().getDate()
        exit_str = str(gfov_exit).replace("T", " ==> ")
        exit_list.append(exit_str)

        gfov_duration = gfov_exit.durationFrom(gfov_entry) / 60  # Convert to minutes
        duration_list.append(gfov_duration)

# Create the DataFrame
df_gfov = pd.DataFrame({
    'GFoV Entry': entry_list,
    'GFoV Exit': exit_list,
    'Duration (min)': duration_list
})

# Display the DataFrame
display(df_gfov)

,GFoV Entry,GFoV Exit,Duration (min)
0,2024-09-22 ==> 12:22:16.7602692952411Z,2024-09-22 ==> 12:23:26.77008289320816Z,1.166830
1,2024-09-22 ==> 20:32:10.54497706141908Z,2024-09-22 ==> 20:32:26.37315697084701Z,0.263803
2,2024-09-23 ==> 19:44:16.43115428025628Z,2024-09-23 ==> 19:44:34.15813385891295Z,0.295450


In [23]:
ele_events = ele_logger.getLoggedEvents()            

entry_list = []
exit_list = []
duration_list = []

for event in ele_events: 
    if event.isIncreasing():
        ele_entry = event.getState().getDate()
        entry_str = str(ele_entry).replace("T", " ==> ")
        entry_list.append(entry_str)
    else:
        ele_exit = event.getState().getDate()
        exit_str = str(ele_exit).replace("T", " ==> ")
        exit_list.append(exit_str)

        ele_duration = ele_exit.durationFrom(ele_entry) / 60  # Convert to minutes
        duration_list.append(ele_duration)

# Create the DataFrame
df_ele = pd.DataFrame({
    'Elevation Entry': entry_list,
    'Elevation Exit': exit_list,
    'Duration (min)': duration_list
})

# Display the DataFrame
display(df_ele)

,Elevation Entry,Elevation Exit,Duration (min)
0,2024-09-21 ==> 11:29:52.7322863211689Z,2024-09-21 ==> 11:38:42.84936712718175Z,8.835285
1,2024-09-21 ==> 13:05:23.34223815300618Z,2024-09-21 ==> 13:16:05.23463129668192Z,10.698207
2,2024-09-21 ==> 14:44:19.63687762658984Z,2024-09-21 ==> 14:52:04.50429488369723Z,7.747790
3,2024-09-21 ==> 16:25:14.86425787278222Z,2024-09-21 ==> 16:28:07.60465952542764Z,2.879007
4,2024-09-21 ==> 18:02:24.72290857986265Z,2024-09-21 ==> 18:08:22.44343196576673Z,5.962009
5,2024-09-21 ==> 19:38:13.45250164809309Z,2024-09-21 ==> 19:48:19.90131318141713Z,10.107480
6,2024-09-21 ==> 21:14:50.40036221079698Z,2024-09-21 ==> 21:25:18.33011066577278Z,10.465496
7,2024-09-22 ==> 10:43:17.36339353909758Z,2024-09-22 ==> 10:50:01.03391247100525Z,6.727842
8,2024-09-22 ==> 12:17:28.60202575653934Z,2024-09-22 ==> 12:28:17.0892277610721Z,10.808120
9,2024-09-22 ==> 13:55:39.31682951955587Z,2024-09-22 ==> 14:04:27.91849634950566Z,8.810028


In [24]:
comb_events = combined_logger.getLoggedEvents()            

entry_list = []
exit_list = []
duration_list = []

for event in comb_events: 
    if event.isIncreasing():
        comb_entry = event.getState().getDate()
        entry_str = str(comb_entry).replace("T", " ==> ")
        entry_list.append(entry_str)
    else:
        comb_exit = event.getState().getDate()
        exit_str = str(comb_exit).replace("T", " ==> ")
        exit_list.append(exit_str)

        comb_duration = comb_exit.durationFrom(comb_entry) / 60  # Convert to minutes
        duration_list.append(comb_duration)

# Create the DataFrame
df_comb = pd.DataFrame({
    'Combined Entry': entry_list,
    'Combined Exit': exit_list,
    'Duration (min)': duration_list
})

# Display the DataFrame
display(df_comb)

,Combined Entry,Combined Exit,Duration (min)
0,2024-09-22 ==> 12:22:16.7602692952411Z,2024-09-22 ==> 12:23:26.77008289320816Z,1.166830
1,2024-09-22 ==> 20:32:10.54497706141908Z,2024-09-22 ==> 20:32:26.37315697084701Z,0.263803
2,2024-09-23 ==> 19:44:16.43115428025628Z,2024-09-23 ==> 19:44:34.15813385891295Z,0.295450
